In [43]:
# import langid
# with open("./dataset/daily_dialogue.txt", "r") as f:
#     lines = f.readlines()
#     print(len(lines))
#     lines = [line for line in lines if langid.classify(line)[0] != 'zh']
#     print(len(lines))

# with open("./dataset/daily_dialogue.txt", "w") as f:
#     f.writelines(lines)

In [44]:
# train a bert tokenizer
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer()

tokenizer.train(
    files=["./dataset/daily_dialogue.txt"],
    vocab_size=30000,
    min_frequency=3,
)

tokenizer.save_model("./bert-babylm-demo")

print(f"vocab size: {tokenizer.get_vocab_size()}")




vocab size: 13600


In [45]:
from datasets import load_dataset
dataset = load_dataset("text", data_files={"train": ["./dataset/daily_dialogue.txt", "./dataset/bnc_spoken.txt"]}, split="train").train_test_split(test_size=0.1, shuffle=True, seed=42)
train_dataset, test_dataset = dataset["train"], dataset["test"]

In [46]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("./bert-babylm-demo", max_len=512)

train_dataset = dataset['train'].map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length'), batched=True)
test_dataset = dataset['test'].map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length'), batched=True)

Map:   0%|          | 0/116027 [00:00<?, ? examples/s]

Map:   0%|          | 0/12892 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertConfig, BertForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

config = BertConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

model = BertForMaskedLM(config=config)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./Bert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=1e-4,
    weight_decay=0.01,
    report_to="none",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #prediction_loss_only=True,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./bert-babylm")